<a href="https://colab.research.google.com/github/Luceliafn/Alura/blob/Machine-Learning-Introdu%C3%A7%C3%A3o-a-classifica%C3%A7%C3%A3o-com-SKLearn/Machine_Learning_Valida%C3%A7%C3%A3o_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Módulo 2 KFold e Aleatoriedade

Ainda que nosso algorítimo esteja funcionando, vamos revisar o código e nos atentar aos seus detalhes:

In [9]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados

x = dados [['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados ['vendido']

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)


Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [8]:
#aleatoriedade no cross validate
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


O que desejamos modificar é a aleatoriedade, nós desejamos que ele embaralhe nossos dados, entretanto, não existe um parâmetro para isto. Todavia, o cv aceita como parâmetro, tanto números, quanto geradores de validação cruzada. E existem diversos geradores de validação cruzada. O que estamos usando é o KFold, que "corta" nossos dados em K pedaços. Por sorte, o sklearn tem o KFold, no modo selection:

class sklearn.model_selection.KFold(nsplits=’warn’, shuffle=False, randomstate=None)

É ele que estamos usando por baixo dos panos, ou seja, sorrateiramente. Então, por enquanto, nosso cross_validate é um KFold de n_splits, no caso, igual a 10. Vamos rodar o código, que com essas modificações, ficou da seguinte forma:

In [20]:
#def imprime_resultados(results):
 #   media = results['test_score'].mean()
 #  desvio_padrao = results['test_score'].std()
 #  print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [46]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f][%.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Assim, ela recebe os results inteiros e, em seguida faz a média, o desvio padrão e imprime os resultado. Neste caso, podemos copiar e colar esta parte do código.

Para não dependermos de uma variável extra, excluímos 10 de print(). E, no código acima, adicionaremos uma linha ao fim do código, para que o algorítimo chame-se apenas imprime_resultados() de modo que não tenhamos de fazer manualmente sempre que formos rodar o código. Ele diretamente pegará a média, o desvio padrão e os imprimirá.

In [47]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados (results)

Accuracy médio: 75.76
Accuracy intervalo: [73.26][78.26]


Como retorno, obteremos:

Accuracy com cross validation, 10 = [74.37, 77.19]COPIAR CÓDIGO
Agora, o ideal é pegarmos os valores da média de desvio padrão e colocarmos em uma função, para facilitarmos nosso trabalho. Nossa função será definida como:

Podemos subir o código de imprime_resultados() para cima, selecionando-o e pressionando o botão "CELL (com a seta para cima)", para que a definição venha em primeiro. A ordenação ficará da seguinte forma:

E rodemos o código logo acima novamente, da mesma forma que estávamos fazendo, com KFold de n_splits igual a 10. Como retorno, obteremos:

Accuracy com cross validation = [74.37, 77.19]


Repare como ficou difícil de acharmos a média. Por esta razão, vamos adicionar um print() para ela em imprime_resultados(). Para especificar, mudaremos os nomes dos prints para Accuracy médio e Accuracy intervalo, respectivamente.

É importante ressaltar que se rodarmos diversas vezes, Accuracy se mantém, pois continua sendo determinístico. Mas ainda assim, é diferente dos valores apresentados sem o shuffle. No entanto, repare que a média obtida por meio dos dois algorítimos é bastante próxima, mas o intervalo é diferente. Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo. Esta é a forma tradicional de execução com shuffle e n_splits.

Assim, podemos usar não somente o KFold, mas, em seu Model Selection, encontraremos o KFold e diversos outros "quebradores" de grupos, que são as Splitter classes. Estudaremos alguns destes outros adiante.